In [1]:
from __future__ import division
import matplotlib 
# matplotlib.use('Qt5Agg') - интерактивные графики в отдельном окне \
# с использованием Qt
%matplotlib qt 
# интерактивные графики в отдельном окне (другой способ)
import matplotlib.pyplot as plt                              
import numpy as np
import time
import numba as nb
import math as mt
from scipy import interpolate as intp
import sys

matplotlib.rcParams.update({'font.size': 25})

# Определения функций для воротных переменных

In [57]:
#@nb.jit(nopython=True)
def alpha_n(V): return 0.01 * (V + 55.0) / (1.0 - np.exp(-(V + 55.0) / 10.0))

#@nb.jit(nopython=True)
def beta_n(V):  return 0.125 * np.exp(-(V + 65.0) / 80.0)

#@nb.jit(nopython=True)
def n_inf(V): return alpha_n(V) / (alpha_n(V) + beta_n(V))

#@nb.jit(nopython=True)
def alpha_m(V): return 0.1*(V + 40.0)/(1.0 - np.exp(-(V + 40.0)/10.0))

#@nb.jit(nopython=True)
def beta_m(V): return 4.0*np.exp(-(V + 65.0) / 18.0)

#@nb.jit(nopython=True)
def m_inf(V): return alpha_m(V) / (alpha_m(V) + beta_m(V))

#@nb.jit(nopython=True)
def alpha_h(V): return 0.07*np.exp(-(V + 65.5) / 20.0)

#@nb.jit(nopython=True)
def beta_h(V): return 1.0 / (1.0 + np.exp(-(V + 35.5) / 10.0))

#@nb.jit(nopython=True)
def h_inf(V): return alpha_h(V) / (alpha_h(V) + beta_h(V))

# Определения функций для оценки поргешности численного решения

In [3]:
def CalculateNorm(analiticalSolutionFunc, numericalSolutionArr, T, dt, amplitude):

    # calculating RRHS error
    numerator, denumenator = 0, 0
    tArray = np.arange(0, T + dt, dt)
    
    deltaArray = np.fabs(analiticalSolutionFunc(tArray) - numericalSolutionArr[:])
    numerator = np.sum(deltaArray**2)
    denumenator = np.sum((analiticalSolutionFunc(tArray))**2)

    RRHSerror = np.sqrt(numerator / denumenator)

    # calculating max|o| error
    maxModError = np.amax(deltaArray)
    errorsList = [100 * RRHSerror, 100*maxModError / amplitude]

    # allowing spline-interpolation to happen when measuring errors
    for i in range(len(errorsList)):
        if np.isnan(errorsList[i]) == True:
            errorsList[i] = sys.float_info.max
    
    return errorsList

# Функции правых частей для воротных переменных и мембранного потенциала

In [25]:
#g_n = 120

#@nb.jit(nopython=True)
def rhs(i_s, m_, n_, h_, v_):
    g_n = 800.
    g_k = 36.
    g_l = 0.3
    v_n = 50.0 # "true" Nernst potential = 50.0
    v_k = -77.0 # "true" Nernst potential = -77.0
    v_l = -54.387 # "true" Nernst potential = -54.387
    c = 1
    return (1. / c) * (i_s - g_n * (m_) ** 3 * (h_) * (v_ - v_n) - g_k * (n_ ** 4) * (v_ - v_k) - g_l * (v_ - v_l))

#@nb.jit(nopython=True)
def rhs_gating_vars(alpha_, beta_, v_, var_):
    return alpha_(v_) * (1 - var_) - beta_(v_) * var_



# functions, performing timestepping
#@nb.jit((nb.float64[:], nb.int64, nb.float64, nb.int64, \
#                        nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], \
#                        nb.float64[:], nb.float64), nopython=True)
def CalculateHHusingExplicitEuler(time_array, size, dt, counter, m, n, h, v, Rhs, I_s, v_rest):
    m[0] = m_inf(v_rest)
    h[0] = h_inf(v_rest)
    n[0] = n_inf(v_rest)
    v[0] = v_rest

    for i in range(1, size):
        m[i] = m_inf(v[i-1]) + (m[i-1] - m_inf(v[i-1])) * np.exp(-dt * (alpha_m(v[i-1]) + beta_m(v[i-1])))
        n[i] = n_inf(v[i-1]) + (n[i-1] - n_inf(v[i-1])) * np.exp(-dt * (alpha_n(v[i-1]) + beta_n(v[i-1])))
        h[i] = h_inf(v[i-1]) + (h[i-1] - h_inf(v[i-1])) * np.exp(-dt * (alpha_h(v[i-1]) + beta_h(v[i-1])))

        Rhs[i-1] = rhs(I_s[i-1], m[i-1], n[i-1], h[i-1], v[i-1])

        dv = Rhs[i-1] * dt
        v[i] = v[i-1] + dv



#@nb.jit((nb.float64[:], nb.int64, nb.float64, nb.int64, \
#                        nb.float64[:], nb.float64[:], nb.float64[:], \
#         nb.float64[:], nb.float64[:], nb.float64[:], nb.float64), nopython=True)
def CalculateHHusingSImplicitEuler(time_array, size, dt, counter, m, n, h, v, Rhs, I_s, v_rest):


    m[0] = m_inf(v_rest)
    h[0] = h_inf(v_rest)
    n[0] = n_inf(v_rest)
    v[0] = v_rest

    step = 1e-4 # step for calculating f'(V) via finite difference

    for i in range(1, size):
        m[i] = m_inf(v[i-1]) + (m[i-1] - m_inf(v[i-1])) * np.exp(-dt * (alpha_m(v[i-1]) + beta_m(v[i-1])))
        n[i] = n_inf(v[i-1]) + (n[i-1] - n_inf(v[i-1])) * np.exp(-dt * (alpha_n(v[i-1]) + beta_n(v[i-1])))
        h[i] = h_inf(v[i-1]) + (h[i-1] - h_inf(v[i-1])) * np.exp(-dt * (alpha_h(v[i-1]) + beta_h(v[i-1])))

        Rhs[i-1] = rhs(I_s[i-1], m[i-1], n[i-1], h[i-1], v[i-1])
        derivative = (rhs(I_s[i-1], m[i-1], n[i-1], h[i-1], v[i-1] + step) - Rhs[i-1]) / step

        dv = Rhs[i-1] * dt / (1 - dt * derivative)
        v[i] = v[i-1] + dv

# Параметры расчетной сетки + др. константы

In [26]:
T = 8 # endtime in msec
numBlocksAnalitical = 2**17
dtAnalitical = T / numBlocksAnalitical # must be less than 10e-4
numBlocksArray = np.array([int(2**n) for n in range(5, 17)])
dtArray = np.asarray([dtAnalitical] + list(float(T)/numBlocksArray[::-1])) #[1e-5] + [0.009232069419105898, 0.0041864984602040045] #np.array([1e-5] + list(float(T)/numPointsArray[::-1]))
print('List of timesteps: ', dtArray)

step_v = 1e-4
d_gating_var = 1e-7
v0 = -65

error, error_e, timeMaxError, timeMaxError_e = [], [], [], []
amplitude = 0
counter = 0
omega = 1 # relaxation parameter

List of timesteps:  [6.10351562e-05 1.22070312e-04 2.44140625e-04 4.88281250e-04
 9.76562500e-04 1.95312500e-03 3.90625000e-03 7.81250000e-03
 1.56250000e-02 3.12500000e-02 6.25000000e-02 1.25000000e-01
 2.50000000e-01]


#Функция "main"

In [27]:
counter = 0
analiticalSolutionFunc = None
amplitude = 0
analiticalSolution = None
start = time.clock()
listOfTimeArrays, listOfNumericalSolutions = list(), list()
error, error_e = [], []
timingEE, timingSIE = [], []

for dt in dtArray:
    startMine = time.clock()

    print ('Iteration #%d' % counter)
    time_array = np.arange(0, T + dt, dt)
    SIZE = len(time_array)
    v = 0*np.ones(SIZE)
    m = np.zeros(SIZE)
    n = np.zeros(SIZE)
    h = np.zeros(SIZE)
    Rhs = np.zeros(SIZE) # [0. for i in range(size)]
    v_e = 0 * np.ones(SIZE) #[0. for i in range(size)]
    m_e = np.zeros(SIZE) #[0. for i in range(size)]
    h_e = np.zeros(SIZE) # [0. for i in range(size)]
    n_e = np.zeros(SIZE) #[0. for i in range(size)]
    Rhs_e = np.zeros(SIZE) #[0. for i in range(size)]
    I_s = np.zeros(SIZE) #[10. for i in range(size)]
    I_s[:] = 10. # original value = 10


    NUM_LAUNCHES = 1 # N = 10 is used for the article
    startEE = time.clock()
    for i in range(NUM_LAUNCHES):
        CalculateHHusingExplicitEuler(time_array, SIZE, dt, counter, m, n, h, v_e, Rhs, I_s, v0)
    timingEE.append((time.clock() - startEE)/NUM_LAUNCHES)

    startSIE = time.clock()
    for i in range(NUM_LAUNCHES):
        CalculateHHusingSImplicitEuler(time_array, SIZE, dt, counter, m, n, h, v, Rhs, I_s, v0)
    timingSIE.append((time.clock() - startSIE)/NUM_LAUNCHES)


    if counter == 0:
        analiticalSolution = np.zeros(SIZE)
        analiticalSolution[:] = v_e[:] # correct is v_e[:] instead of v[:], switch later
        MIN, MAX = np.amin(analiticalSolution), np.amax(analiticalSolution)
        amplitude = MAX - MIN
        analiticalSolution[:] -= MIN
        analiticalSolutionFunc = intp.interp1d(time_array, analiticalSolution)


    timeMine = time.clock() - startMine

    v[:] -= MIN
    v_e[:] -= MIN

    errTmp = CalculateNorm(analiticalSolutionFunc, v, T, dt, amplitude)
    errTmp_e = CalculateNorm(analiticalSolutionFunc, v_e, T, dt, amplitude)
     
    error.append(errTmp)
    error_e.append(errTmp_e)

    listOfTimeArrays.append(time_array)
    listOfNumericalSolutions.append(v_e)
    counter += 1

print ('Elapsed time = %.2e sec' % (time.clock() - start))

Iteration #0


Iteration #1


Iteration #2


Iteration #3


Iteration #4


Iteration #5


Iteration #6


Iteration #7
Iteration #8


Iteration #9
Iteration #10
Iteration #11
Iteration #12
Elapsed time = 4.51e+01 sec


/Users/quothewhite/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


# Таблица погрешностей, графики погрешностей

In [29]:
import pandas as pd

error = np.asarray(error)
error_e = np.asarray(error_e)

print(error[:,0])
#print(len(dtArray), ' ', len(error_e[:,0]))


listOfTolerances = [1., 3., 5.]
LEN = len(listOfTolerances)
NUM_ERROR_TYPES = 2

errorFunctionRRMS = intp.interp1d(error[:,0], dtArray)
errorFunctionMaxmod = intp.interp1d(error[:,1], dtArray)
timingFunctionRRMS = intp.interp1d(error[:,0], timingSIE)
timingFunctionMaxmod = intp.interp1d(error[:,1], timingSIE)

error_e_FunctionRRMS = intp.interp1d(error_e[:,0], dtArray)
error_e_FunctionMaxmod = intp.interp1d(error_e[:,1], dtArray)
timing_e_FunctionRRMS = intp.interp1d(error_e[:,0], timingEE)
timing_e_FunctionMaxmod = intp.interp1d(error_e[:,1], timingEE)


d = {('dt', 'Явный метод'): pd.Series(list(error_e_FunctionRRMS(listOfTolerances)) + 
                                list(error_e_FunctionMaxmod(listOfTolerances)),
                            index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]]),
     ('dt', 'Неявный метод'): pd.Series(list(errorFunctionRRMS(listOfTolerances)) + 
                            list(errorFunctionMaxmod(listOfTolerances)),
                            index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]]),
     ('t_calc', 'Явный метод'): pd.Series(list(timing_e_FunctionRRMS(listOfTolerances)) + 
                                list(timing_e_FunctionMaxmod(listOfTolerances)),
                            index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]]),
     ('t_calc', 'Неявный метод'): pd.Series(list(timingFunctionRRMS(listOfTolerances)) + 
                            list(timingFunctionMaxmod(listOfTolerances)),
                            index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]])
   }


df = pd.DataFrame(d)
print(df)

writer = pd.ExcelWriter('hh_model_g_na_120.xlsx')
df.to_excel(writer, 'Sheet1')
writer.save()

[1.00402980e-02 3.73615471e-02 9.23929472e-02 2.02300159e-01
 4.21160620e-01 8.54878482e-01 1.70577328e+00 3.33936883e+00
 6.33918814e+00 1.14520500e+01 1.94581764e+01 3.16409389e+01
 4.80978664e+01]
                    dt                    t_calc            
         Неявный метод Явный метод Неявный метод Явный метод
RRMS   1      0.002286    0.003465      0.331251    0.261877
       3      0.007001    0.007812      0.117114    0.092845
       5      0.012137    0.007812      0.070054    0.092845
Maxmod 1      0.000478    0.000706      1.500910    1.124527
       3      0.001369    0.002003      0.604355    0.393904
       5      0.002282    0.003318      0.331557    0.275167


In [30]:
matplotlib.rcParams.update({'font.size': 25})

plt.figure(figsize=(13,8.5))
plt.loglog((timingSIE[1:]), (error[1:,0]), 'k-o', label='Упрощенный неявный метод', linewidth=4, markersize=15)
plt.loglog((timingEE[1:]), (error_e[1:,0]), 'k-^', label='Явный метод', linewidth=4, markersize = 15)
plt.grid('on')
plt.xlabel('Время вычисления, с')
plt.ylabel('Погрешность RRMS, %')
plt.ylim([1e-2, 1e2])
plt.axhline(y=5, linewidth=4, color='k', linestyle='--', label='5%')
plt.axhline(y=1, linewidth=4, color='k', linestyle='-', label='1%')
plt.legend(loc='best', fontsize=20)

plt.figure(figsize=(13,8.5))
plt.loglog((timingSIE[1:]), (error[1:,1]), 'k-o', label='Упрощенный неявный метод', linewidth=4, markersize = 15)
plt.loglog((timingEE[1:]), (error_e[1:,1]), 'k-^', label='Явный метод', linewidth=4, markersize = 15)
plt.grid('on')
plt.xlabel('Время вычисления, с')
plt.ylabel('Погрешность Maxmod, %')
plt.ylim([1e-2, 1e2])
plt.axhline(y=5, linewidth=4, color='k', linestyle='--', label='5%')
plt.axhline(y=1, linewidth=4, color='k', linestyle='-', label='1%')
plt.legend(loc='best', fontsize=20)

/Users/quothewhite/anaconda/envs/python3/lib/python3.6/site-packages/numpy/ma/core.py:6641: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)


#Оценка шагов по времени, при которых достигается заданная точность

In [15]:
tolerance2 = 5.
dtArrayForTolerance = [float(errorFunctionRRMS(tolerance2)), float(errorFunctionMaxmod(tolerance2))]
print(dtArrayForTolerance)

[0.009215357607712735, 0.004270490964415012]


In [16]:
counter = 0
analiticalSolutionFunc = None
amplitude = 0
analiticalSolution = None
MIN, MAX = 0, 0
start = time.clock()
listOfTimeArrays, listOfNumericalSolutions = list(), list()

dtArray2 = [dtAnalitical] + dtArrayForTolerance

for dt in dtArray2:
    startMine = time.clock()

    print ('Iteration #%d' % counter)
    time_array = np.arange(0, T + dt, dt)
    SIZE = len(time_array)
    v = 0*np.ones(SIZE)
    m = np.zeros(SIZE)
    n = np.zeros(SIZE)
    h = np.zeros(SIZE)
    Rhs = np.zeros(SIZE) # [0. for i in range(size)]
    v_e = 0 * np.ones(SIZE) #[0. for i in range(size)]
    m_e = np.zeros(SIZE) #[0. for i in range(size)]
    h_e = np.zeros(SIZE) # [0. for i in range(size)]
    n_e = np.zeros(SIZE) #[0. for i in range(size)]
    Rhs_e = np.zeros(SIZE) #[0. for i in range(size)]
    I_s = np.zeros(SIZE) #[10. for i in range(size)]
    I_s[:] = 10. # original value = 10


    
    CalculateHHusingExplicitEuler(time_array, SIZE, dt, counter, m, n, h, v_e, Rhs, I_s, v0)
    CalculateHHusingSImplicitEuler(time_array, SIZE, dt, counter, m, n, h, v, Rhs, I_s, v0)
    


    if counter == 0:
        analiticalSolution = np.zeros(SIZE)
        analiticalSolution[:] = v_e[:] # correct is v_e[:] instead of v[:], switch later
        MIN, MAX = np.amin(analiticalSolution), np.amax(analiticalSolution)
        amplitude = MAX - MIN
        analiticalSolution[:] -= MIN
        analiticalSolutionFunc = intp.interp1d(time_array, analiticalSolution)


    timeMine = time.clock() - startMine

    v[:] -= MIN
    v_e[:] -= MIN

    listOfTimeArrays.append(time_array)
    listOfNumericalSolutions.append(v)
    counter += 1

Iteration #0


Iteration #1
Iteration #2


In [23]:
plt.figure(figsize=(13, 8.5))

plt.plot(listOfTimeArrays[0], analiticalSolution, 'k-', label='Референсное решение', linewidth=2)
plt.plot(listOfTimeArrays[1], listOfNumericalSolutions[1], 'k--', label='Погрешность 5% по норме RRMS', linewidth=2)    
plt.plot(listOfTimeArrays[2], listOfNumericalSolutions[2], 'k:', label='Погрешность 5% по норме Maxmod', linewidth=2)
plt.legend(loc='best', prop={'size': 20})
plt.xlabel('Время, мс')
plt.ylabel('V, мВ')
#plt.ylim([-10, 140])
plt.xlim([0, 8])
plt.grid('on')
plt.show()

# Построение графиков

In [28]:
plt.figure(figsize=(13, 8.5))

tAnalyticalArray = listOfTimeArrays[0]
plt.plot(tAnalyticalArray, analiticalSolution, 'k-', \
         label=r'$\mathregular{g_{Na} = %d \, мСм/см^2}$' % 800, lw=4)
    
plt.legend(loc='best')
plt.xlabel('Время, мс')
plt.ylabel('V, мВ')
#plt.ylim([0, 140])
plt.xlim([0., T])
plt.grid('on')
plt.show()

## Комментарии:

* За аналитическое можно принять решение при dt = 1e-4 мс.
* Порог возбуждения принимаем равным $V_{trh} = -55 \, мВ$, для всех значений проводимости $g_{Na}$



###Пересчитать потенциал ГХК при увеличении проводимостей! Потенциал покоя примерно одинаковый при всех $g_{Na} = 120 - 200 \,$, и равен $V_{rest} = -65 \, мВ$. При $g_{Na} >= 200$ (примерно 200) - система начинает "автоколебания", стационарного состояния не существует; тем не менее, начальное условие для всех проводимостей $g_{Na}$ устанавливаем $V_0 = V_{rest} = -65 \, мВ$, для единообразия.

Потенциал покоя $V_{rest} = -65 \, мВ$ при $g_{Na} = 120$.

# Графики производных и их _Max_ значения

### NB: При вычислении производных высших порядков возникают огромные ошибки (вроде, это ошибки округления судя по "пилам" на графиках); возможно подумать, как это исправить.

In [41]:
import scipy.interpolate as interpol
milli = 1e-3 # msec -> sec; UPDATE: transition to msec is not needed


VAnalyticalArray = analiticalSolution

tAnalyticalArray = listOfTimeArrays[0] # corresponding array of timesteps, in msec
dt = np.diff(tAnalyticalArray)[0] # [0]=[1]=[2]=...=[n] because the grid is equally spaced

matplotlib.rcParams.update({'font.size': 20})
xTicks = [0, 4, 8]
#from matplotlib.ticker import FormatStrFormatter
# plotting derivatives calculated via finite-difference formulas
maxOrderDerivative = 5
valuesArray, tArray, dt = VAnalyticalArray, tAnalyticalArray, np.diff(tAnalyticalArray)[0]
plt.figure(figsize=(13, 8.5))

# plotting analytical solution
ax = plt.subplot(2, 3, 1)
plt.plot(tArray[1:], valuesArray[1:], 'k-', lw=3)  
plt.grid('on')
plt.xlim([0, 8])
ax.set_xticks(xTicks)
ax.set_xticklabels([])
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
plt.ylabel(r'$\mathregular{V, мВ}$')   

# plotting derivatives
for i in range(1, maxOrderDerivative + 1):
    ax = plt.subplot(2, 3, i + 1)
    
    dt = np.diff(tArray)[0]
    derivativeArray = np.gradient(valuesArray, dt)
    plt.plot(tArray[2:-2], derivativeArray[2:-2], 'k-', lw=3)  
    plt.grid('on')
    plt.xlim([0, 8])
    ax.set_xticks(xTicks)
    ax.set_xticklabels([])
    #ax.yaxis.set_major_formatter(FormatStrFormatter('%.1e'))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
    
    plt.ylabel(r'$\mathregular{V^{\left(%d\right)}, мВ/мс^{%d}}$' % (i, i))
    
    if i > 1:
        ax.set_title(r'$\mathregular{\max |V^{\left(%d\right)}| = %.2e \; мВ/мс^{%d}}$' \
                    % (i, np.fabs(derivativeArray[2:-2]).max(), i), fontsize=15, y = 1.15)
    
    if i > 2: # set xlabels for bottom subplots
        ax.set_xticks(xTicks)
        ax.set_xticklabels([0, 4, 8])
        plt.xlabel(r'Время, мс')
    
    tArray = tArray[::2]
    valuesArray = derivativeArray[::2]
    
plt.show()

# Графики аналитических решений при различных $g_{Na}$

In [31]:
#@nb.jit(nopython=True)
def rhs(i_s, m_, n_, h_, v_, g_n):
    g_k = 36
    g_l = 0.3
    v_n = 50.0 # "true" Nernst potential = 50.0
    v_k = -77.0 # "true" Nernst potential = -77.0
    v_l = -54.387 # "true" Nernst potential = -54.387
    c = 1
    return (1. / c) * (i_s - g_n * (m_) ** 3 * (h_) * (v_ - v_n) - g_k * (n_ ** 4) * (v_ - v_k) - g_l * (v_ - v_l))




# functions, performing timestepping
#@nb.jit((nb.float64[:], nb.int64, nb.float64, \
#                        nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], \
#                        nb.float64[:], nb.float64, nb.float64), nopython=True)
def CalculateHHusingExplicitEuler(time_array, size, dt,  m, n, h, v, Rhs, I_s, v_rest, g_n):
    m[0] = m_inf(v_rest)
    h[0] = h_inf(v_rest)
    n[0] = n_inf(v_rest)
    v[0] = v_rest

    for i in range(1, size):
        m[i] = m_inf(v[i-1]) + (m[i-1] - m_inf(v[i-1])) * np.exp(-dt * (alpha_m(v[i-1]) + beta_m(v[i-1])))
        n[i] = n_inf(v[i-1]) + (n[i-1] - n_inf(v[i-1])) * np.exp(-dt * (alpha_n(v[i-1]) + beta_n(v[i-1])))
        h[i] = h_inf(v[i-1]) + (h[i-1] - h_inf(v[i-1])) * np.exp(-dt * (alpha_h(v[i-1]) + beta_h(v[i-1])))

        Rhs[i-1] = rhs(I_s[i-1], m[i-1], n[i-1], h[i-1], v[i-1], g_n)

        dv = Rhs[i-1] * dt
        v[i] = v[i-1] + dv

In [38]:
counterOuter = 0
g_n = [80., 120.] + [float(2*i) for i in range(100, 401, 100)]
dt = dtAnalitical # timestep
print('dt = %.2e' % dt)
T = 8. # endtime
v_0 = -65.

# grid arrays and parameters 
time_array = np.arange(0, T + dt, dt)
SIZE = len(time_array)
v = 0*np.ones(SIZE)
m = np.zeros(SIZE)
n = np.zeros(SIZE)
h = np.zeros(SIZE)
Rhs = np.zeros(SIZE)
v_e = 0 * np.ones(SIZE)
m_e = np.zeros(SIZE)
h_e = np.zeros(SIZE)
n_e = np.zeros(SIZE)
Rhs_e = np.zeros(SIZE)
I_s = np.zeros(SIZE)
I_s[:] = 10.
analiticalSolutionsList = []

for stiffnessParameter in g_n: 
    print('Iteration #%d' % counterOuter)
    
    CalculateHHusingExplicitEuler(time_array, SIZE, dt, m, n, h, v_e, Rhs, I_s, v_0, stiffnessParameter)
    
    analiticalSolution = np.zeros(len(time_array))
    analiticalSolution[:] = v_e[:]
    MIN, MAX = np.amin(analiticalSolution), np.amax(analiticalSolution)
    amplitude = MAX - MIN
    analiticalSolution[:] -= MIN

    counterOuter += 1
    analiticalSolutionsList.append(analiticalSolution)
    
# plotting analytical solutions
lineStylesTmp = [':', '-', '--', '-.', ':', '-']
lineStyles = [('k' + styleTmp) for styleTmp in lineStylesTmp]

plt.figure()
matplotlib.rcParams.update({'font.size': 25})
for i in range(len(g_n)):
        plt.plot(time_array, analiticalSolutionsList[i], lineStyles[i], label=r'$\mathregular{g_{Na} = %d \, мСм/см^2}$' % g_n[i], linewidth=4)
        plt.legend(loc='best', fontsize=20)
        plt.xlabel(r'Время, мс')
        plt.ylabel(r'V, мВ')
        #plt.ylim([0, 140])
        plt.xlim([0, 8])
        plt.grid('on')
plt.show()

dt = 6.10e-05
Iteration #0


Iteration #1


Iteration #2


Iteration #3


Iteration #4


Iteration #5


# Ускорение при различных величинах $g_{Na}$

In [59]:
#@nb.jit(nopython=True)
def rhs(i_s, m_, n_, h_, v_, g_n):
    g_k = 36
    g_l = 0.3
    v_n = 50.0 # "true" Nernst potential = 50.0
    v_k = -77.0 # "true" Nernst potential = -77.0
    v_l = -54.387 # "true" Nernst potential = -54.387
    c = 1
    return (1. / c) * (i_s - g_n * (m_) ** 3 * (h_) * (v_ - v_n) - g_k * (n_ ** 4) * (v_ - v_k) - g_l * (v_ - v_l))

#@nb.jit(nopython=True)
def rhs_gating_vars(alpha_, beta_, v_, var_):
    return alpha_(v_) * (1 - var_) - beta_(v_) * var_



# functions, performing timestepping
#@nb.jit((nb.float64[:], nb.int64, nb.float64, nb.int64, \
#                        nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], \
 #                       nb.float64[:], nb.float64, nb.float64), nopython=True)
def CalculateHHusingExplicitEuler(time_array, size, dt, counter, m, n, h, v, Rhs, I_s, v_rest, g_n):
    m[0] = m_inf(v_rest)
    h[0] = h_inf(v_rest)
    n[0] = n_inf(v_rest)
    v[0] = v_rest

    for i in range(1, size):
        m[i] = m_inf(v[i-1]) + (m[i-1] - m_inf(v[i-1])) * np.exp(-dt * (alpha_m(v[i-1]) + beta_m(v[i-1])))
        n[i] = n_inf(v[i-1]) + (n[i-1] - n_inf(v[i-1])) * np.exp(-dt * (alpha_n(v[i-1]) + beta_n(v[i-1])))
        h[i] = h_inf(v[i-1]) + (h[i-1] - h_inf(v[i-1])) * np.exp(-dt * (alpha_h(v[i-1]) + beta_h(v[i-1])))

        Rhs[i-1] = rhs(I_s[i-1], m[i-1], n[i-1], h[i-1], v[i-1], g_n)

        dv = Rhs[i-1] * dt
        v[i] = v[i-1] + dv



#@nb.jit((nb.float64[:], nb.int64, nb.float64, nb.int64, \
         #               nb.float64[:], nb.float64[:], nb.float64[:], \
         #nb.float64[:], nb.float64[:], nb.float64[:], nb.float64,
         #nb.float64), nopython=True)
def CalculateHHusingSImplicitEuler(time_array, size, dt, counter, m, n, h, v, Rhs, I_s, v_rest, g_n):


    m[0] = m_inf(v_rest)
    h[0] = h_inf(v_rest)
    n[0] = n_inf(v_rest)
    v[0] = v_rest

    step = 1e-4 # step for calculating f'(V) via finite difference

    for i in range(1, size):
        m[i] = m_inf(v[i-1]) + (m[i-1] - m_inf(v[i-1])) * np.exp(-dt * (alpha_m(v[i-1]) + beta_m(v[i-1])))
        n[i] = n_inf(v[i-1]) + (n[i-1] - n_inf(v[i-1])) * np.exp(-dt * (alpha_n(v[i-1]) + beta_n(v[i-1])))
        h[i] = h_inf(v[i-1]) + (h[i-1] - h_inf(v[i-1])) * np.exp(-dt * (alpha_h(v[i-1]) + beta_h(v[i-1])))

        Rhs[i-1] = rhs(I_s[i-1], m[i-1], n[i-1], h[i-1], v[i-1], g_n)
        derivative = (rhs(I_s[i-1], m[i-1], n[i-1], h[i-1], v[i-1] + step, g_n) - Rhs[i-1]) / step

        dv = Rhs[i-1] * dt / (1 - dt * derivative)
        v[i] = v[i-1] + dv

In [60]:
g_n_array = [80., 120.] + [float(2*i) for i in range(100, 401, 100)]
numPointsArray = np.array([int(2**n) for n in range(5, 17)])
T = 8.
dtArray = np.asarray([dtAnalitical] + list(float(T)/numPointsArray[::-1])) 

print('Timesteps: ', dtArray)

Timesteps:  [6.10351562e-05 1.22070312e-04 2.44140625e-04 4.88281250e-04
 9.76562500e-04 1.95312500e-03 3.90625000e-03 7.81250000e-03
 1.56250000e-02 3.12500000e-02 6.25000000e-02 1.25000000e-01
 2.50000000e-01]


In [67]:
counterInner, counterOuter = 0, 0
speedupRRMS, speedupMaxmod = [], []
start = time.clock()

for g_n in g_n_array:
    print('\nCalclulating for gNa = %.2f mSm/cm^2' % g_n)
    timingEE, timingSIE = [], []
    error, error_e = [], []
    counterInner = 0
    for dt in dtArray:
        startMine = time.clock()

        print ('     Iteration #%d' % counterInner)
        time_array = np.arange(0, T + dt, dt)
        SIZE = len(time_array)
        v = 0*np.ones(SIZE)
        m = np.zeros(SIZE)
        n = np.zeros(SIZE)
        h = np.zeros(SIZE)
        Rhs = np.zeros(SIZE) # [0. for i in range(size)]
        v_e = 0 * np.ones(SIZE) #[0. for i in range(size)]
        m_e = np.zeros(SIZE) #[0. for i in range(size)]
        h_e = np.zeros(SIZE) # [0. for i in range(size)]
        n_e = np.zeros(SIZE) #[0. for i in range(size)]
        Rhs_e = np.zeros(SIZE) #[0. for i in range(size)]
        I_s = np.zeros(SIZE) #[10. for i in range(size)]
        I_s[:] = 10


        NUM_LAUNCHES = 10
        startEE = time.clock()
        for i in range(NUM_LAUNCHES):
            CalculateHHusingExplicitEuler(time_array, SIZE, dt, counter, m, n, h, v_e, Rhs, I_s, v0, g_n)        
        timingEE.append((time.clock() - startEE)/NUM_LAUNCHES)

        startSIE = time.clock()
        for i in range(NUM_LAUNCHES):
            CalculateHHusingSImplicitEuler(time_array, SIZE, dt, counter, m, n, h, v, Rhs, I_s, v0, g_n)
        timingSIE.append((time.clock() - startSIE)/NUM_LAUNCHES)

        if counterInner == 0:
            analiticalSolution = np.zeros(SIZE)
            analiticalSolution[:] = v_e[:]
            MIN, MAX = np.amin(analiticalSolution), np.amax(analiticalSolution)
            amplitude = MAX - MIN
            analiticalSolution[:] -= MIN
            analiticalSolutionFunc = intp.interp1d(time_array, analiticalSolution)


        timeMine = time.clock() - startMine

        v[:] -= MIN
        v_e[:] -= MIN

        errTmp = CalculateNorm(analiticalSolutionFunc, v, T, dt, amplitude)
        errTmp_e = CalculateNorm(analiticalSolutionFunc, v_e, T, dt, amplitude)

        error.append(errTmp)
        error_e.append(errTmp_e)

        counterInner += 1

    counterOuter += 1
    error_for_calc = np.array(error)
    error_e_for_calc = np.array(error_e)

    # DEBUG: print(error_e_for_calc)
    
    speedupRRMS.append([intp.interp1d(error_e_for_calc[:, 0], timingEE[:])(5.) / \
                       intp.interp1d(error_for_calc[:, 0], timingSIE[:])(5.), \
                        intp.interp1d(error_e_for_calc[:, 0], timingEE[:])(1.) / \
                        intp.interp1d(error_for_calc[:, 0], timingSIE[:])(1.)]
                       )


    speedupMaxmod.append([intp.interp1d(error_e_for_calc[:, 1], timingEE[:])(5.) / \
                       intp.interp1d(error_for_calc[:, 1], timingSIE[:])(5.), \
                          intp.interp1d(error_e_for_calc[:, 1], timingEE[:])(1.) / \
                          intp.interp1d(error_for_calc[:, 1], timingSIE[:])(1.)]
                         )

print ('time elapsed = %.2e sec' % (time.clock() - start))

speedupRRMS = np.array(speedupRRMS)
speedupMaxmod = np.array(speedupMaxmod)


Calclulating for gNa = 80.00 mSm/cm^2
     Iteration #0


     Iteration #1
     Iteration #2


     Iteration #3
     Iteration #4
     Iteration #5
     Iteration #6
     Iteration #7
     Iteration #8
     Iteration #9
     Iteration #10
     Iteration #11
     Iteration #12

Calclulating for gNa = 120.00 mSm/cm^2
     Iteration #0


     Iteration #1
     Iteration #2


     Iteration #3
     Iteration #4
     Iteration #5
     Iteration #6
     Iteration #7
     Iteration #8
     Iteration #9
     Iteration #10
     Iteration #11
     Iteration #12

Calclulating for gNa = 200.00 mSm/cm^2
     Iteration #0


...

     Iteration #2
     Iteration #3
     Iteration #4
     Iteration #5
     Iteration #6


     Iteration #7
     Iteration #8
     Iteration #9
     Iteration #10
     Iteration #11
     Iteration #12

Calclulating for gNa = 800.00 mSm/cm^2
     Iteration #0


     Iteration #1


     Iteration #2
     Iteration #3
     Iteration #4
     Iteration #5
     Iteration #6


     Iteration #7
     Iteration #8
     Iteration #9
     Iteration #10
     Iteration #11
     Iteration #12
time elapsed = 5.42e+00 sec


In [68]:
plt.figure(figsize=(13, 8.5))
lw_ = 2
plt.plot(g_n_array, speedupRRMS[:, 0], 'k-o', label=r'5% RRMS', lw=lw_, 
         markersize=10, fillstyle='none')
plt.plot(g_n_array, speedupRRMS[:, 1], 'k-.^', label=r'1% RRMS', lw=lw_, 
         markersize=10, fillstyle='none')
plt.plot(g_n_array, speedupMaxmod[:, 0], 'k--s', label=r'5% Maxmod', lw=lw_, 
         markersize=10, fillstyle='none')
plt.plot(g_n_array, speedupMaxmod[:, 1], 'k:v', label=r'1% Maxmod', lw=lw_, 
         markersize=10, fillstyle='none')
plt.legend(loc='best')

plt.xlabel(r'$\mathregular{g_{Na}, \, мСм/см^2}$')
plt.ylabel(r'Ускорение')
plt.ylim([0.4, 1.5])
plt.grid('on')
plt.show()